In [21]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.1,
)

In [22]:
import requests
import tempfile

data_url = "https://gist.githubusercontent.com/serranoarevalo/5acf755c2b8d83f1707ef266b82ea223/raw/d72b9558a11523adbe13300b41321ecd93d331d3/document.txt"

response = requests.get(data_url)

with tempfile.NamedTemporaryFile(delete=False, suffix=".txt") as temp_file:
    temp_file.write(response.content)
    temp_file_path = temp_file.name
    
print(temp_file_path)

/var/folders/b1/wt2bjx71325fk62cxg92dwl80000gn/T/tmplmssu8yz.txt


In [23]:
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

memory = ConversationBufferMemory(
    llm=llm,
    max_token_limit=80,
    return_messages=True
)

loader = UnstructuredFileLoader(file_path=temp_file_path)
docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)
vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        ("human", "{question}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = ({
            "context": retriever, 
            "question": RunnablePassthrough(),
        } 
         | prompt 
         | llm
)

def get_cache_data(question):
    loaded_data = load_memory(_)
    print(loaded_data)
    for i in range(0, len(loaded_data), 2):
        human = loaded_data[i]
        if human.content == question:
            hit = loaded_data[i + 1]
            print("memory history hit!!!")
            return hit 
    return None
    
def invoke_chain(question):
    result = get_cache_data(question)
    if result is None:
        result = chain.invoke(question)
        memory.save_context(
            {"input": question},
            {"output": result.content},
        )
    print("[Answer]", result)


In [24]:
invoke_chain("Is Aaronson guilty?")

[]
[Answer] content="According to the context, Winston believes that Aaronson, along with Jones and Rutherford, is guilty of the crimes they were charged with, despite having a memory that contradicts this belief. He concludes that he had never seen the photograph that disproved their guilt and that it had never existed, indicating that he has accepted the Party's version of reality." response_metadata={'token_usage': <OpenAIObject at 0x103673cb0> JSON: {
  "prompt_tokens": 2333,
  "completion_tokens": 70,
  "total_tokens": 2403
}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_f33667828e', 'finish_reason': 'stop', 'logprobs': None} id='run-176d422b-358c-443e-a491-8431338c057b-0'


In [25]:
invoke_chain("What message did he write on the table?")

[HumanMessage(content='Is Aaronson guilty?'), AIMessage(content="According to the context, Winston believes that Aaronson, along with Jones and Rutherford, is guilty of the crimes they were charged with, despite having a memory that contradicts this belief. He concludes that he had never seen the photograph that disproved their guilt and that it had never existed, indicating that he has accepted the Party's version of reality.")]
[Answer] content='He traced with his finger in the dust on the table: 2+2=5.' response_metadata={'token_usage': <OpenAIObject at 0x313c80110> JSON: {
  "prompt_tokens": 2318,
  "completion_tokens": 19,
  "total_tokens": 2337
}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_f905cf32a9', 'finish_reason': 'stop', 'logprobs': None} id='run-e853c311-634f-4cb4-a5f9-0f47614c6622-0'


In [26]:
invoke_chain("Who is Julia?")

[HumanMessage(content='Is Aaronson guilty?'), AIMessage(content="According to the context, Winston believes that Aaronson, along with Jones and Rutherford, is guilty of the crimes they were charged with, despite having a memory that contradicts this belief. He concludes that he had never seen the photograph that disproved their guilt and that it had never existed, indicating that he has accepted the Party's version of reality."), HumanMessage(content='What message did he write on the table?'), AIMessage(content='He traced with his finger in the dust on the table: 2+2=5.')]


Retrying langchain_community.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-embedding-ada-002 in organization org-DxyR7ijEF9ww6YcVJ24D6t40 on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..


RateLimitError: Rate limit reached for text-embedding-ada-002 in organization org-DxyR7ijEF9ww6YcVJ24D6t40 on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.